In [ ]:
#%matplotlib

In [1]:
import shapefile
import numpy as np

import sys,os
import h5py

print (os.environ["SEACAS_DIR"])
#sys.path.append('/Users/ajc/Core/SimDataInputs/ats-repo/ats/tools/meshing_ats/')
sys.path.append(os.path.join(os.environ['ATS_SRC_DIR'],'tools','meshing_ats','meshing_ats'))

import meshing_ats

from matplotlib import pyplot as plt
from matplotlib import collections


#%matplotlib inline

eps = 1.0

/Users/ajc/codes/simulator/debug/ats-mesh/tools/seacas


In [2]:
def dist(p0,p1):
    return np.linalg.norm(p0.x - p1.x)

def is_equal(p0,p1,eps=1.0):
    if (dist(p0,p1) < eps):
        return True
    return False

def centroid(points):
    return Point(sum(p.x for p in points)/len(points))

def ccw(A, B, C):
    """Tests whether the turn formed by A, B, and C is ccw"""
    return (B.x[0] - A.x[0]) * (C.x[1] - A.x[1]) > (B.x[1] - A.x[1]) * (C.x[0] - A.x[0])

def convex(points):
    """Returns 1 if convex, ccw ordered, -1 if convex but cw ordered, 0 if not convex"""
    is_ccw = []
    for i in range(len(points)):
        # Check every triplet of points
        A = points[i % len(points)]
        B = points[(i + 1) % len(points)]
        C = points[(i + 2) % len(points)]
        is_ccw.append(ccw(A,B,C))
    if all(is_ccw):
        return 1
    elif all([not(c) for c in is_ccw]):
        return -1
    return 0    

def star_convex(points):
    """Returns 1 if star-convex, ccw ordered, -1 if star-convex but cw ordered, 0 if not star-convex"""
    cent = centroid(points)
    convex_tris = []
    for i in range(len(points)):
        tri_points = [points[i], points[(i+1)%len(points)], cent]
        convex_tris.append(convex(tri_points))
    if all([cv_tri == 1 for cv_tri in convex_tris]):
        return 1
    elif all([cv_tri == -1 for cv_tri in convex_tris]):
        return -1
    return 0    

class Point(object):
    def __init__(self, x, decimals=1):
        self.x = np.array(x)
        self.xr = np.array([np.round(self.x[0],decimals), 
                            np.round(self.x[1],decimals)])
        
    def __hash__(self):
        return hash(tuple(self.xr))
        
    def __eq__(self, other):
        return is_equal(self, other, eps)
    
    def __repr__(self):
        return "Point: (%12.10g,%12.10g)"%(self.xr[0],self.xr[1])




In [3]:
# read the polygons via shapefile
sf_poly = shapefile.Reader("./Polygon_mesh_final/Polygon_mesh_final")
polygons = sf_poly.shapes()

nodes = []
conn = []

for s in polygons:
    poly_conn = []
    for p in s.points:
        myp = Point(p)
        if len(nodes) > 0:
            i = np.argmin(np.array([dist(myp, lcv) for lcv in nodes]))

            min_dist = dist(myp, nodes[i])
            if min_dist > eps:
                poly_conn.append(len(nodes))
                nodes.append(myp)
            else:
                poly_conn.append(i)
        else:
            nodes.append(myp)
            poly_conn.append(0)
    conn.append(poly_conn)
    
print (len(nodes))

xyz_nodes = np.zeros((len(nodes),3),'d')
for i,n in enumerate(nodes):
    xyz_nodes[i,0:2] = n.x

959


In [4]:
# write the nodes file
with open("./Polygon_mesh_nodes_final.csv",'w') as fid:
    fid.write("point ID,X,Y\n")
    for i,n in enumerate(nodes):
        fid.write("%d,%16.16g,%16.16g\n"%(i,n.x[0],n.x[1]))


In [5]:
# read the resulting shapefile with x,y,z nodes
sf_points = shapefile.Reader("./Polygon_mesh_nodes_sample_final/Polygon_mesh_nodes_sample_final")
shapes = sf_points.shapes()

xyz_points = np.array([[p.points[0][0], p.points[0][1], p.z[0]] for p in shapes])
points = [Point(xyz_points[i,0:2]) for i in range(xyz_points.shape[0])]

# match nodes to xyz_points
for i,n in enumerate(nodes):
    j_closest = np.argmin(np.array([dist(n,p) for p in points]))
    xyz_nodes[i,2] = xyz_points[j_closest,2]


In [6]:
# potentially strip duplicate points
for c in conn:
    if c[0] == c[-1]:
        c.pop()

In [7]:
%matplotlib 
# make the 2D mesh
import color
m2 = meshing_ats.Mesh2D(xyz_nodes,conn)
m2.plot(color=['b','r','g','m','c'])
plt.show()

Using matplotlib backend: MacOSX


In [8]:
# check for non-star-convex cells
non_star_convex = []
for i,c in enumerate(conn):
    points = [nodes[j] for j in c]
    cv = star_convex(points)
    if cv == -1:
        c.reverse()
    elif cv == 1:
        # pass
        0 == 0
    else:
        print ("Non-star-convex cell:", i, c)
        plt.figure()
        c2 = c[:]
        c2.append(c2[0])
        coords = xyz_nodes[c2]
        plt.plot(coords[:,0],coords[:,1], 'b-x')
        cent = centroid(points)
        plt.scatter([cent.x[0],], [cent.x[1],], s=50, marker='s', c='r')
        non_star_convex.append(i)

fig,ax = plt.subplots(1,1)

m2.plot(color=['b','r','g','m','c'],ax=ax)

verts = [[xyz_nodes[i,0:2] for i in conn[p]] for p in non_star_convex]
gons = collections.PolyCollection(verts, facecolors='r')
ax.add_collection(gons)

plt.show()

In [9]:
#Use VisIt to get IDs of High-centered polygons, the GIDs should be obtained in parallel runs
path = '/Users/ajc/research/PreProcessing/python-scripts/meshes/watershed-barrow/'
def read_poly_ids():
    ids = []
    with open(os.path.join(path,'poly_char_gids/manually_char_polygons.txt')) as f:
        for line in f:
            if 'Zone:' in line:
                c = int(line.split()[1])
                if not c in ids:
                    ids.append(c)
    return ids


def get_ponded_depth():
    d1 = h5py.File('/Users/ajc/Projects/ATS-Data/OR-CONDO/ats-intermediate-ngee/surface_only/Rain_R1-serial/visdump_surface_data.h5','r')
    var = 'surface-ponded_depth.cell.0'
    keys = np.array(d1[var].keys(), 'i')
    keys = np.sort(keys, axis=None)
    cycle = '%s'%keys[-1]
    Ids_pd = d1[var][cycle][:]
    
    return Ids_pd
    

In [10]:
#IDS1= read_poly_ids()

In [11]:
IDS2= read_poly_ids()

In [12]:
#id1=np.sort(IDS1)
hcp_gids=np.sort(IDS2)
#print (id1, hcp_gids)
#print (len(id1), len(hcp_gids))

In [13]:
#Changing organic layer thickness 
def org_layer_bottom_bndry(polygon, id):
    if (polygon == 'HCP' and id % 2 == 0):
        thickness = 0.1 
    elif(polygon == 'HCP'):
        thickness = 0.06 
    elif (polygon == 'LCP' and id %3 == 0):
        thickness = 0.2
    else:
        thickness = 0.16
    return thickness

def org_layer_bottom_bndry_gids(id):
    if id in hcp_gids:
        thickness = 0.08
    else:
        thickness= 0.16
        
    return thickness


def org_layer_bottom_bndry_pd(pd):
    pd = pd[0] * 100;
    #print pd
    if pd < 0.5:
        thickness = 8.
    elif (pd > 5.):
        thickness = 20
    elif (pd >=0.5 and pd<=5.0):
        m = (20. - 8.)/(5.- 0.5)
        y = 8. + m*(pd - 0.5)
        thickness = np.floor(y) - np.floor(y)%2
    return thickness/100.

In [14]:
#Ids = read_poly_ids()
#ponded_depth = get_ponded_depth()
gid_based = True

peat_thickness = np.zeros(m2.num_cells())

if gid_based == True:
    for i in range(m2.num_cells()):
        if i in hcp_gids:
            peat_thickness[i] = org_layer_bottom_bndry_gids(i)
        else:
            peat_thickness[i] = org_layer_bottom_bndry_gids(i)
else:
    for i in range(m2.num_cells()):
        peat_thickness[i] = org_layer_bottom_bndry_pd(ponded_depth[i])
        print (i, peat_thickness[i], ponded_depth[i]*100)

print (peat_thickness)

[0.08 0.08 0.08 0.08 0.08 0.08 0.16 0.08 0.08 0.16 0.16 0.16 0.16 0.16
 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.08 0.08
 0.16 0.16 0.08 0.08 0.08 0.08 0.16 0.08 0.08 0.16 0.08 0.08 0.16 0.16
 0.16 0.16 0.08 0.08 0.16 0.16 0.08 0.16 0.08 0.08 0.16 0.16 0.08 0.08
 0.08 0.08 0.08 0.08 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16
 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.08
 0.16 0.08 0.16 0.16 0.08 0.08 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.08
 0.16 0.16 0.16 0.16 0.08 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.08
 0.16 0.16 0.16 0.16 0.16 0.08 0.16 0.16 0.16 0.08 0.08 0.08 0.08 0.08
 0.16 0.08 0.16 0.08 0.16 0.08 0.16 0.08 0.16 0.16 0.16 0.08 0.16 0.16
 0.08 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16
 0.16 0.16 0.16 0.16 0.16 0.08 0.08 0.08 0.16 0.16 0.16 0.16 0.16 0.16
 0.08 0.16 0.08 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16 0.16
 0.16 0.16 0.08 0.08 0.16 0.16 0.08 0.08 0.16 0.16 0.08 0.16 0.16 0.08
 0.08 

In [15]:
# Ice rich zone 50 cm
#variable peat thickness
outfile = path
# layer extrusion
layer_types = []
layer_data = []
layer_ncells = []
layer_mat_ids = []


z=0
Z = []
 

for i in range(8):#10 
    layer_types.append('constant')
    layer_data.append(0.02)
    layer_ncells.append(1)
    layer_mat_ids.append(-10000*np.ones((m2.num_cells(),),'i'))
    z = round(z + 0.02, 6)
    Z.append(z)
print ('Peat ', z)

for i in range(17): #8cm peat, n=20, 20cm peat n = 14
    layer_types.append('constant')
    layer_data.append(0.02)
    layer_ncells.append(1)
    layer_mat_ids.append(1003*np.ones((m2.num_cells(),),'i'))
    z = round(z + 0.02, 6)
    Z.append(z)
print ('Upper mineral ', z)


dz = .02
for i in range(30):
    dz *= 1.075
    layer_types.append("constant")
    layer_data.append(dz)
    layer_ncells.append(1)
    layer_mat_ids.append(1004*np.ones((m2.num_cells(),),'i'))
    z = round(z + dz,6)
    Z.append(z)
print ('Ice rich', z)

for i in range(25):
    dz *= 1.14
    layer_types.append("constant")
    layer_data.append(dz)
    layer_ncells.append(1)
    layer_mat_ids.append(1005*np.ones((m2.num_cells(),),'i'))
    z = round(z + dz,6)
    Z.append(z)
print (z)



layer_types.append('snapped')
layer_data.append(-45.0) # bottom location
layer_ncells.append(1)
layer_mat_ids.append(1005*np.ones((m2.num_cells(),),'i'))


mat_ids=np.zeros((m2.num_cells(), 9), 'i')
#mat_ids=np.zeros((m2.num_cells(), 11), 'i')

for i in range(m2.num_cells()):
    for j in range(9):
        if j == 0 :
            mat_ids[i,j]=1001
        elif (Z[j] <= peat_thickness[i]):
            #print (i,j,Z[j],peat_thickness[i])
            mat_ids[i,j]=1002
        else:
            mat_ids[i,j]=1003
    #break
for j in range(9):
    layer_mat_ids[j] = mat_ids[:,j]

print ('HERE ',len(layer_mat_ids), len(layer_ncells), len(layer_types), len(layer_data))
#print (Z)
#print (layer_mat_ids[8])

m3 = meshing_ats.Mesh3D.extruded_Mesh2D(m2, layer_types, 
                                        layer_data, 
                                        layer_ncells, 
                                        layer_mat_ids)
print ('testing.exo'.split("."))
#m3.write_exodus("testing.exo")#"barrow_polygon_watershed_5layer-icerich50cm-omvar.exo")
#m3.write_exodus(os.path.join(outfile + 'barrow_polygon_watershed_5layer-icerich50cm-omvar.exo').encode('utf8'))
m3.write_exodus('barrow_polygon_watershed_5layer-icerich50cm-omvar.exo'.encode('utf8'))

Peat  0.16
Upper mineral  0.5
Ice rich 2.723088
39.026866
HERE  81 81 81 81
['testing', 'exo']

You are using exodus.py v 1.16 (seacas-py3), a python wrapper of some of the exodus library.

Copyright (c) 2013, 2014, 2015, 2016, 2017, 2018, 2019 National Technology &
Engineering Solutions of Sandia, LLC (NTESS).  Under the terms of
Contract DE-NA0003525 with NTESS, the U.S. Government retains certain
rights in this software.

BASENAME_FILE_Exodus:  barrow_polygon_watershed_5layer-icerich50cm-omvar.exo
barrow_polygon_watershed_5layer-icerich50cm-omvar
Opening exodus file: barrow_polygon_watershed_5layer-icerich50cm-omvar.exo
Closing exodus file: barrow_polygon_watershed_5layer-icerich50cm-omvar.exo


In [ ]:
#Old 40cm ice rich zone
outfile = path
# layer extrusion
layer_types = []
layer_data = []
layer_ncells = []
layer_mat_ids = []

z=0
Z = []

for i in range(1):
    layer_types.append('constant')
    layer_data.append(0.02)
    layer_ncells.append(1)
    layer_mat_ids.append(1001)
    z = z + 0.02
    Z.append(z)
print ('Moss ', z)

for i in range(4): 
    layer_types.append('constant')
    layer_data.append(0.02)
    layer_ncells.append(1)
    layer_mat_ids.append(1002)
    z = z + 0.02
    Z.append(z)
print ('Peat ', z)  
for i in range(15): #8cm peat, n=20, 20cm peat n = 14
    layer_types.append('constant')
    layer_data.append(0.02)
    layer_ncells.append(1)
    layer_mat_ids.append(1003)
    z = z + 0.02
    Z.append(z)
print ('Upper mineral ', z)


dz = .02
for i in range(35):
    dz *= 1.075
    layer_types.append("constant")
    layer_data.append(dz)
    layer_ncells.append(1)
    layer_mat_ids.append(1004)
    z = z + dz
    Z.append(z)
print ('Ice rich', z)

for i in range(25):
    dz *= 1.12
    layer_types.append("constant")
    layer_data.append(dz)
    layer_ncells.append(1)
    layer_mat_ids.append(1005)
    z = z + dz
    Z.append(z)
print (z)


layer_types.append('snapped')
layer_data.append(-45.0) # bottom location
layer_ncells.append(1)
layer_mat_ids.append(1005)

#print len(Z)

print Z
#print layer_data
m3 = meshing_ats.Mesh3D.extruded_Mesh2D(m2, layer_types, 
                                        layer_data, 
                                        layer_ncells, 
                                        layer_mat_ids)

m3.write_exodus(outfile + "barrow468-5layers-om10cm-rich.exo")

In [ ]:
# For 55 cm ice rich zone
outfile = "/Users/ajc/Desktop/ATS/meshes/watershed-barrow/"
# layer extrusion
layer_types = []
layer_data = []
layer_ncells = []
layer_mat_ids = []

z=0
Z = []

for i in range(1):
    layer_types.append('constant')
    layer_data.append(0.02)
    layer_ncells.append(1)
    layer_mat_ids.append(1001)
    z = z + 0.02
    Z.append(z)
print ('Moss ', z)

for i in range(4): 
    layer_types.append('constant')
    layer_data.append(0.02)
    layer_ncells.append(1)
    layer_mat_ids.append(1002)
    z = z + 0.02
    Z.append(z)
print ('Peat ', z)  
for i in range(15): #8cm peat, n=20, 20cm peat n = 14
    layer_types.append('constant')
    layer_data.append(0.02)
    layer_ncells.append(1)
    layer_mat_ids.append(1003)
    z = z + 0.02
    Z.append(z)
print ('Upper mineral ', z)


dz = .02
for i in range(4):
    dz *= 1.075
    layer_types.append("constant")
    layer_data.append(dz)
    layer_ncells.append(1)
    layer_mat_ids.append(1003)
    z = z + dz
    Z.append(z)
print ('Ice rich', z)

#dz = .02
for i in range(31):
    dz *= 1.075
    layer_types.append("constant")
    layer_data.append(dz)
    layer_ncells.append(1)
    layer_mat_ids.append(1004)
    z = z + dz
    Z.append(z)
print ('Ice rich', z)

for i in range(25):
    dz *= 1.12
    layer_types.append("constant")
    layer_data.append(dz)
    layer_ncells.append(1)
    layer_mat_ids.append(1005)
    z = z + dz
    Z.append(z)
print (z)


layer_types.append('snapped')
layer_data.append(-45.0) # bottom location
layer_ncells.append(1)
layer_mat_ids.append(1005)

#print len(Z)

print Z
#print layer_data
m3 = meshing_ats.Mesh3D.extruded_Mesh2D(m2, layer_types, 
                                        layer_data, 
                                        layer_ncells, 
                                        layer_mat_ids)

m3.write_exodus(outfile + "barrow468-5layers-om10cm-rich.exo")